# Setu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/GenAIFinalProject/

In [ ]:
root = "/content/drive/MyDrive/GenAIFinalProject/"

In [ ]:
!pip install -r /content/drive/MyDrive/GenAIFinalProject/requirements.txt
!pip install importnb
!pip install wandb
!pip install import-ipynb


In [ ]:
!pip install ipynb

In [ ]:
import os
import itertools
import numpy as np
import argparse
import gdown
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import pytorch_lightning as pl
import os.path as osp
import wandb
import gc

from tqdm import tqdm
from torchmetrics.image import PeakSignalNoiseRatio
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger


from resnet import resnet34
from attention import AttentionStack, LayerNorm, AddBroadcastPosEmbed
from utils import shift_dim
from importnb import imports
import import_ipynb

from videotransformerdataset import VideoDataset

from vq_vae import VQVAE


In [ ]:
import torch.utils.data as data
import glob
import os.path as osp
import torch
import pytorch_lightning as pl
import os
from torch.utils.data import SubsetRandomSampler
from pytorch_lightning.callbacks import ModelCheckpoint

# Custom dataset class
class ClipDataset(data.Dataset):

    def __init__(self, root):

        super().__init__()
        self.items = []
        files = glob.glob(osp.join(root, '**', f'*.pt'), recursive=True)

        print(f"Loading {len(files)} stores")
        for file in files:
          vals = torch.load(file).to(torch.float)
          clips = torch.unbind(vals)
          self.items += clips
        self.sequence_length = self.items[0].shape[1]
        self.resolution = self.items[0].shape[2]

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

# Adapter for torch lightning
class VideoData(pl.LightningDataModule):

    def __init__(self, root):
        super().__init__()
        self.train_root = osp.join(root, "Train")
        self.test_root = osp.join(root, "Test")

    def _dataloader(self, files):
        dataset = ClipDataset(files)
        dataloader = data.DataLoader(
            dataset,
            batch_size=4,
            num_workers=16,
            pin_memory=True,
            sampler=SubsetRandomSampler(torch.randint(high=len(dataset), size=(int(0.1 * len(dataset)),)))
        )
        return dataloader

    def train_dataloader(self):
        return self._dataloader(self.train_root)

    def val_dataloader(self):
        return self._dataloader(self.test_root)

    def test_dataloader(self):
        return self.val_dataloader()

# Imported Code
This code is copied directly from https://github.com/wilson1yan/VideoGPT

In [ ]:
def download(id, fname, root=os.path.expanduser('~/.cache/videogpt')):
    os.makedirs(root, exist_ok=True)
    destination = os.path.join(root, fname)

    if os.path.exists(destination):
        return destination

    gdown.download(id=id, output=destination, quiet=False)
    return destination

def load_vqvae(model_name, device=torch.device('cpu'), root=os.path.expanduser('~/.cache/videogpt')):
          filepath = download(_VQVAE[model_name], model_name, root=root)
          vqvae = VQVAE.load_from_checkpoint(filepath).to(device)
          vqvae.eval()
          return vqvae

In [ ]:
class VideoGPT(pl.LightningModule):
    def __init__(self, args):
        super().__init__()
        self.args = args

        # Load VQ-VAE and set all parameters to no grad
        if not os.path.exists(args.vqvae):
            self.vqvae = load_vqvae(args.vqvae)
        else:
            self.vqvae =  VQVAE.load_from_checkpoint(args.vqvae)
        for p in self.vqvae.parameters():
            p.requires_grad = False
        self.vqvae.codebook._need_init = False
        self.vqvae.eval()

        # ResNet34 for frame conditioning
        self.use_frame_cond = args.n_cond_frames > 0
        if self.use_frame_cond:
            frame_cond_shape = (args.n_cond_frames,
                                args.resolution // 4,
                                args.resolution // 4,
                                60)
            self.resnet = resnet34(1, (1, 4, 4), resnet_dim=60)
            self.cond_pos_embd = AddBroadcastPosEmbed(
                shape=frame_cond_shape[:-1], embd_dim=frame_cond_shape[-1]
            )
        else:
            frame_cond_shape = None

        # VideoGPT transformer
        # print(vars(self.vqvae))
        self.shape = (self.args.sequence_length, self.args.resolution // 4,
                       self.args.resolution // 4)

        self.fc_in = nn.Linear(self.vqvae.embedding_dim, args.hidden_dim, bias=False)
        self.fc_in.weight.data.normal_(std=0.02)

        self.attn_stack = AttentionStack(
            self.shape, args.hidden_dim, args.heads, args.layers, args.dropout,
            args.attn_type, args.attn_dropout, args.class_cond_dim, frame_cond_shape
        )

        self.norm = LayerNorm(args.hidden_dim, args.class_cond_dim)

        self.fc_out = nn.Linear(args.hidden_dim, self.vqvae.n_codes, bias=False)
        self.fc_out.weight.data.copy_(torch.zeros(self.vqvae.n_codes, args.hidden_dim))

        # caches for faster decoding (if necessary)
        self.frame_cond_cache = None

        self.save_hyperparameters()

    def get_reconstruction(self, videos):
        return self.vqvae.decode(self.vqvae.encode(videos))

    def sample(self, n, batch=None):
        device = self.fc_in.weight.device

        cond = dict()
        if self.use_frame_cond or self.args.class_cond:
            assert batch is not None
            video = batch['video']

            if self.args.class_cond:
                label = batch['label']
                cond['class_cond'] = F.one_hot(label, self.args.class_cond_dim).type_as(video)
            if self.use_frame_cond:
                cond['frame_cond'] = video[:, :, :self.args.n_cond_frames]

        samples = torch.zeros((n,) + self.shape).long().to(device)
        idxs = list(itertools.product(*[range(s) for s in self.shape]))

        with torch.no_grad():
            prev_idx = None
            for i, idx in enumerate(tqdm(idxs)):
                batch_idx_slice = (slice(None, None), *[slice(i, i + 1) for i in idx])
                batch_idx = (slice(None, None), *idx)
                embeddings = self.vqvae.codebook.dictionary_lookup(samples)

                if prev_idx is None:
                    # set arbitrary input values for the first token
                    # does not matter what value since it will be shifted anyways
                    embeddings_slice = embeddings[batch_idx_slice]
                    samples_slice = samples[batch_idx_slice]
                else:
                    embeddings_slice = embeddings[prev_idx]
                    samples_slice = samples[prev_idx]

                logits = self(embeddings_slice, samples_slice, cond,
                              decode_step=i, decode_idx=idx)[1]
                # squeeze all possible dim except batch dimension
                logits = logits.squeeze().unsqueeze(0) if logits.shape[0] == 1 else logits.squeeze()
                probs = F.softmax(logits, dim=-1)
                samples[batch_idx] = torch.multinomial(probs, 1).squeeze(-1)

                prev_idx = batch_idx_slice
            samples = self.vqvae.decode(samples)
            samples = torch.clamp(samples, -0.5, 0.5) + 0.5

        return samples # BCTHW in [0, 1]


    def forward(self, x, targets, cond, decode_step=None, decode_idx=None):
        if self.use_frame_cond:
            if decode_step is None:
                cond['frame_cond'] = self.cond_pos_embd(self.resnet(cond['frame_cond']))
            elif decode_step == 0:
                self.frame_cond_cache = self.cond_pos_embd(self.resnet(cond['frame_cond']))
                cond['frame_cond'] = self.frame_cond_cache
            else:
                cond['frame_cond'] = self.frame_cond_cache

        h = self.fc_in(x)
        h = self.attn_stack(h, cond, decode_step, decode_idx)
        h = self.norm(h, cond)
        logits = self.fc_out(h)
        loss = F.cross_entropy(shift_dim(logits, -1, 1), targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        self.vqvae.eval()
        x = batch
        cond = dict()
        if self.args.class_cond:
            label = batch['label']
            cond['class_cond'] = F.one_hot(label, self.args.class_cond_dim).type_as(x)
        if self.use_frame_cond:
            cond['frame_cond'] = x[:, :, :self.args.n_cond_frames]

        with torch.no_grad():
            targets, x = self.vqvae.encode(x, include_embeddings=True)
            x = shift_dim(x, 1, -1)

        loss, _ = self(x, targets, cond)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.training_step(batch, batch_idx)
        self.log('val/loss', loss, prog_bar=True)
        wandb.log({'val/loss': loss})

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-4, betas=(0.9, 0.999))
        assert hasattr(self.args, 'max_steps') and self.args.max_steps is not None, f"Must set max_steps argument"
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, self.args.max_steps)
        return [optimizer], [dict(scheduler=scheduler, interval='step', frequency=1)]


    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--vqvae', type=str, default='kinetics_stride4x4x4',
                            help='path to vqvae ckpt, or model name to download pretrained')
        parser.add_argument('--n_cond_frames', type=int, default=7)
        parser.add_argument('--class_cond', action='store_true')

        # VideoGPT hyperparmeters
        parser.add_argument('--hidden_dim', type=int, default=576)
        parser.add_argument('--heads', type=int, default=4)
        parser.add_argument('--layers', type=int, default=8)
        parser.add_argument('--dropout', type=float, default=0.2)
        parser.add_argument('--attn_type', type=str, default='full',
                            choices=['full', 'sparse'])
        parser.add_argument('--attn_dropout', type=float, default=0.3)

        return parser


# Train Loop

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

wandb.login(key="6839b73984bcab4842ff37c1f24655fdb496167e") #API Key is in your wandb account, under settings (wandb.ai/settings)

class Args(argparse.Namespace):
    data_path = root+'Data'
    vqvae = root+'VideoGPT VQ-VAE/jpn9i0q0/checkpoints/epoch=44-step=29340.ckpt'
    n_cond_frames = 16
    resolution = 64
    sequence_length = 31
    batch_size = 1
    num_workers = 8
    hidden_dim = 288
    max_steps = 100
    max_epochs = 100
    heads = 4
    layers = 5 # 6
    dropout = 0.2
    attn_type = "full"
    attn_dropout = 0.3
    class_cond = None
    gpus = 1

def main():
    pl.seed_everything(1234)
    args = Args()

    data = VideoData(args.data_path)
    # pre-make relevant cached files if necessary
    data.train_dataloader()
    data.test_dataloader()

    args.class_cond_dim = data.n_classes if args.class_cond else None
    model = VideoGPT(args)

    callbacks = []
    callbacks.append(ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=-1))

    kwargs = dict()
    if args.gpus > 1:
        # find_unused_parameters = False to support gradient checkpointing
        kwargs = dict(distributed_backend='ddp', gpus=args.gpus,
                      plugins=[pl.plugins.DDPPlugin(find_unused_parameters=False)])

    wandb_logger = WandbLogger(project="VideoGPT VQ-VAE")

    trainer = pl.Trainer(accelerator="gpu", max_epochs=200, logger = wandb_logger)

    trainer.fit(model, data)


if __name__ == '__main__':
    main()

